# 交叉熵基准测试

*版权所有 (c) 2022 百度量子计算研究所，保留所有权利。*

**交叉熵基准测试（Cross Entropy Benchmarking, XEB）**，是通过搭建不同深度的随机量子电路并多次进行采样计算，拟合采样数据，得到保真度 $F_{XEB}$ 以及含噪电路中的有效噪声率（Effective Noise Rate, ENR），以此来衡量量子设备性能。谷歌使用该方法对自己的 $53$ 量子比特的量子计算硬件悬铃木（Sycamore）进行基准测试,并宣称自己实现了量子霸权[1]。该教程对交叉熵基准测试进行简单的介绍并演示如何在[百度量子平台](https://quantum.baidu.com/)上实现。

## 理论

### 波特-托马斯分布

对于任意一个固定 $n$ 量子比特的比特串 $x^* \in \{0,1\}^{n}$，随机搭建一个 $n$量子比特电路并进行测量，得到该比特串 $x^*$ 出现的概率 $p$。若重复多次搭建不同的随机电路并测量，则可以得到概率 $p$ 的概率，即元概率（meta-probability）。对于性能优越的量子计算机，元概率的概率分布应满足波特-托马斯分布（Porter-Thomas Distribution, PT distribution），即

$$
f(p) = Ne^{-Np},
$$

其中 $N = 2^n$。对于经典计算机，随着量子比特数量的增加，想要模拟出同样的分布结果的复杂度是指数级增加。需要注意的是随机电路的结构以及量子门的选择都会影响电路的随机性，即影响元概率分布的结果以及交叉熵基准测试的结果。

### 随机电路

XEB 中常见的随机电路结构如下图所示[2]

![随机电路](figures/xeb-random-circuit.png "图1：随机量子电路。")

一层单量子比特门接一层双量子比特门的整体，即图中红色方框所示，即为一个循环深度。量易伏中具有实现任意量子比特数、任意循环深度的随机电路的函数。该函数中默认使用的单量子门为（U3）门，双量子比特门默认为（CNOT）门，用户可以自定义不同的量子比特门集合。下面我们引入一些必要的包，并演示实现该函数。

In [ ]:
import numpy as np
from typing import List
import matplotlib.pyplot as plt

from qcompute_qep.utils.circuit import random_circuit, circuit_to_state, print_circuit
from qcompute_qep.benchmarking.xeb import XEB

In [ ]:
qubits=[0, 1]
m = 2 # the depths of cycle
qp = random_circuit(qubits,m)
print_circuit(qp.circuit)

上面我们实现了随机电路。我们可以设计采样，来观察电路最终的结果是否满足 PT 分布。

In [ ]:
def porter_distribution_sample(qubits: List[int], m_cycle: int, trials: int):
    n_qubits = len(qubits)
    dimension = 2 ** n_qubits
    # picks a random bitstring as labelled by the integers 1 to 2**n_qubits

    probs_bitstring = []

    # Start sample
    for i in range(trials):
        env = random_circuit(qubits, m_cycle)
        state = circuit_to_state(env, vector=True)
        state = state.reshape(-1)
        bitstring = np.random.choice(dimension)
        prob = np.abs(state[bitstring]) ** 2
        probs_bitstring.append(prob)
    return probs_bitstring


def plot_sample_result(n_qubits, probs_bitstring):
    dim = 2 ** n_qubits
    xspace = np.linspace(0.0, 1.0, 100)
    yspace = dim * np.exp(-dim * xspace)

    # plot both empirical and theoretical calculations
    plt.figure(figsize=(9, 6))
    plt.hist(probs_bitstring, bins=20, density=True, label='Empirical Distribution')
    plt.plot(xspace, yspace, label='Theoretical Porter-Thomas Distribution')

    plt.xlabel("Probability p")
    plt.ylabel("Probability that the random bistring occurs with probability p")
    plt.legend(loc='best')
    plt.show()

qubits = [0, 1]
N = len(qubits)
M = 10  # number of depth
trials = 1 * 10 ** 4  # number of sample
plot_sample_result(N, porter_distribution_sample(qubits, M, trials))

可以看出函数实现的随机电路大致满足了 PT 分布。电路的随机性除了与量子门和电路结构相关，也与量子比特数和深度相关，读者可以自行修改参数，尝试观察不同的结果。同时也可以固定其他参数，稍微增加 2-3 个量子比特数，观察程序运行时间，就会明白在经典计算机上想要通过足够多的采样模拟该结果是异常困难的。

### 随机基准测试

保真度 $F_{XEB}$ 为真实含噪电路与理论的元概率分布之间的重合程度，上面采样的例子可以看作是完全白噪声的量子电路采样的结果，因为我们对所有结果等概率的采样，其等价于量子电路中最大混态。在这里我们使用线性交叉熵（linear cross entropy）来计算保真度。线性交叉熵公式如下:

$$
\mathcal L\left(P\parallel Q\right) = 2^{n}\sum\limits_{x}P(x)Q(x)-1.
$$

对于保真度 $F_{XEB}$ 的公式如下:

$$
F_{XEB} = \mathcal L\left(P_{noisy}\parallel P_{ideal}\right) \approx \frac{D}{RS}\sum\limits_{r=1}^{R}\sum\limits_{s=1}^{S}P_{ideal}^{(r)}\left(x_{s}^{(r)}\right)-1,
$$

其中 $D = 2^n$，$R$ 为采样的随机电路的数量，$S$ 为每个电路采样的次数，$P_{ideal}^{(r)}$ 为每次随机电路的理想概率，$x_{s}^{(r)}$ 为实际含噪电路 $r$ 层深度的第 $s$ 次测量结果。上述公式中的 $P_{noisy}$ 无法精确的进行数值计算，需要通过大量地采样随机电路来近似。
由于该方法计算的保真度 $F_{XEB}$ 在两个概率分布相等时趋于无穷，所以我们在实际处理中用到无偏线性交叉熵（unbiased linear cross entropy）$F_{uXEB}$

$$
F_{uXEB} = \frac{\frac{D}{RS}\sum_{r,s=1}^{RS}P_{ideal}^{(r)}\left(x_{s}^{(r)}\right)-1} {\frac{D}{R}\sum_{r=1}^{R}{\left(P_{ideal}^{(r)}\left(x^{(r)}\right)\right)}^{2}-1},
$$

$x^{r} \in \{0,1\}^{n}$。当电路为理想电路时，$F_{uXEB}$ 为1。当电路的噪声非常大时，$F_{uXEB}$ 为0。接着我们可以通过不同循环深度电路下的$F_{uXEB}$来计算得到 **ENR**。公式如下:

$$
F = Ae^{-{\lambda}d},
$$

$d$ 为循环深度，$F$ 为 $F_{uXEB}$，$\lambda$ 即为 **ENR**。

## 实践

### 单量子比特电路

In [ ]:
import QCompute

# 请登录“量易伏”平台（https://quantum-hub.baidu.com/）获取 Token
QCompute.Define.hubToken = "TOKEN"

qc = QCompute.BackendName.CloudBaiduQPUQian
xeb_1 = XEB()
xeb_1_results = xeb_1.benchmark(qubits=[0],
                            qc=qc,
                            shots=8192,
                            repeats=20,
                            seq_lengths=[1, 5, 10, 15, 20, 30, 40]
                            )
xeb_1.plot_results()

### 双量子比特电路

在量易伏中我们实现了该方法，可以运行以下代码来查看结果。

In [ ]:
xeb_2 = XEB()
xeb_2_results = xeb_2.benchmark(qubits=[0, 1 ],
                            qc=qc,
                            shots=8192,  
                            repeats=20,
                            seq_lengths=[1, 5, 10, 15, 20, 30, 40]
                            )
xeb_2.plot_results()

上图就是在不同深度下的随机电路采样的数据得到的结果，蓝色为拟合的曲线。通过拟合结果就可以得到 F 以及 ENR。我们可以打印结果进行查看。

In [ ]:
print("The ENR is: ",xeb_1_results['lambda'])
print("The ENR is: ",xeb_2_results['lambda'])

至此我们通过 XEB 基准测试的方法得到了含噪电路中的 **ENR**，并以此衡量了含噪模拟器设备的性能。用户可以通过定义不同的噪声，设置不同的参数来进行实验观察结果。更多的细节感兴趣的读者可以在该论文中查看[2]。

## 参考资料

\[1\] Arute, F., Arya, K., Babbush, R. et al. Quantum supremacy using a programmable superconducting processor. [Nature](https://www.nature.com/articles/s41586-019-1666-5) 574, 505–510 (2019).

\[2\] Liu, Yunchao, et al. "Benchmarking near-term quantum computers via random circuit sampling." [arXiv](https://arxiv.org/abs/2105.05232) preprint arXiv:2105.05232 (2022).



